In [36]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm
from urllib.request import urlopen
import QuantLib as ql

plt.rcParams["figure.figsize"] = (16,9)

**Forward Price(F)**
* mostly traded over the counter

In [50]:
def get_currency_fwd(spot_rate_domestic_forien,domestic_rate_pct,forien_rate_pct,t_years,is_cont=False):
    
    '''
    rate is in perecent
    Replication (Domestic: Yen , Forien: USD)
    1. Borrow Forien: USD 
    2. Sell Forien: USD for Domestic: Yen
    3. Invest the in Domestic: Yen rate
    '''
    # continous case
    if is_cont: return spot_rate_domestic_forien*np.exp((domestic_rate_pct-forien_rate_pct)*t_years/100)

    return spot_rate_domestic_forien*(1+domestic_rate_pct/100)**t_years/(1+forien_rate_pct/100)

def get_fwd_price(spot_price,rf_rate_pct,t_years,is_cont=False):
    
    '''
    rate is in perecent
    Replication 
    1. Borrow USD
    2. Invest the in USD
    '''
    # continous case
    if is_cont: return spot_price*np.exp(rf_rate_pct*t_years/100)

    return spot_price*(1+rf_rate_pct/100)**t_years

In [49]:
get_currency_fwd(120,0.15,2.40,1,True)

117.33014846320036

In [52]:
get_fwd_price(120,1.5,3,True)

125.52334318904603

**Futures Contracts(f)** 

Future(f) = Forward(F) will hold unless


* the riskless rate from now till T is unpredictable
* the spot price S is highly correlate with the rate

In [53]:
def get_future_price(spot_price,rf_rate_pct,t_years,is_cont=False): 
    return get_fwd_price(spot_price,rf_rate_pct,t_years,is_cont)

**Fixed Income Security**

In [55]:
def get_bond_price(coupon_rate, rf_rate, n, face_value):
    return (coupon_rate / rf_rate) * (1 - (1 + rf_rate)**-n) + face_value / (1 + rf_rate)**n

In [60]:
coupon_rate = 5
rf_rate = 0.05
n = 10
face_value = 100
get_bond_price(coupon_rate, rf_rate, n, face_value)

100.0

In [41]:
import QuantLib as ql

today = ql.Date.todaysDate()
calendar = ql.TARGET()
day_counter = ql.ActualActual(ql.ActualActual.ISMA)
issue_date = ql.Date(1, 1, 2021)
maturity_date = ql.Date(1, 1, 2031)
coupon_rate = 0.05
face_value = 100

bond_schedule = ql.Schedule(issue_date, maturity_date, ql.Period(ql.Semiannual), calendar, ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
bond = ql.FixedRateBond(0, face_value, bond_schedule, [coupon_rate], day_counter)

yield_rate = 0.03
discount_curve = ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(yield_rate)), day_counter)
bond.setPricingEngine(ql.DiscountingBondEngine(ql.YieldTermStructureHandle(discount_curve)))

price = bond.dirtyPrice()
print('Bond price:', price)


Bond price: 114.53939534999284


**Swap**
* assume market are perfect


In [ ]:
def get_swap_price(fixed_rate, notional, market_rate, rt ,maturity_years, paid_freq = 2): # maturity_years unit: years
    cash_flows = np.zeros(maturity_years * 12)
    for i in range(maturity_years * 12): # paied everymonth
        if i % 12 == 0:
            cash_flows[i] = -notional * fixed_rate * dt
        else:
            cash_flows[i] = notional * (fixed_rate - market_rate) * dt
    price = np.sum(cash_flows) / (1 + market_rate * dt)**(maturity_years * 12)
    return price

fixed_rate = 0.05
notional = 100
maturity = 1
market_rate = 0.04 # should be the list since it chaning

price = get_swap_price(fixed_rate, notional, maturity, market_rate)
print("Swap price:", price)

In [90]:
def get_fair_swap_fixed_price(discount_rates,fwd_com_prices): # maturity_years unit: years
    discount_rates = np.array(discount_rates)
    fwd_com_prices = np.array(fwd_com_prices)
    return (discount_rates@fwd_com_prices.T)/np.sum(discount_rates)

discount_rates = [0.970,0.941,0.912,0.883]
fwd_com_prices = [1435,1485,1540,1565] # fwd gold price

price = get_fair_swap_fixed_price(discount_rates, fwd_com_prices)
print("Fair fixed swap price:", price) 

Swap price: 1504.508904479223


In [1]:
def get_fair_int_swap_rate(rates,discount_rates):
    """
    Replicate (floating rate using t bill and fixed rate(swap rate))
    1. borrow 1-B0 cash and short bond B0 and invest 1 in tbill at t0
    2. get 1+R12 from t bill, pay 1 back from borrowing cost at t2
    ...
    5. get 1+R45 from t bill, pay 1 back from borrowing cost at t5 
    """
    return (discount_rates@rates.T)/np.sum(discount_rates) # similar to vanilla swap


**CVA: Swaps (Monte-Carlo)**

This will calculate the CVA on a swap with a fixed rate of 5%, a notional amount of $100, a maturity of 1 year, a recovery rate of 40%, a default probability of 10%, and a spread of 1%. The simulation will generate 10,000 scenarios to estimate the expected loss.

In [59]:
def cva_swap(fixed_rate, notional, maturity, recovery_rate, default_prob, spread, num_simulations):
    dt = maturity / num_simulations
    e = np.random.normal(0, 1, num_simulations)
    r = fixed_rate + spread
    pv = np.zeros(num_simulations)
    for i in range(num_simulations):
        if e[i] < default_prob:
            pv[i] = notional * recovery_rate / (1 + r * dt)**(num_simulations - i)
        else:
            pv[i] = notional * fixed_rate * dt / (1 + r * dt)**(num_simulations - i)
    return np.mean(pv)

fixed_rate = 0.05
notional = 100
maturity = 1
recovery_rate = 0.4
default_prob = 0.1
spread = 0.01
num_simulations = 10000

cva = cva_swap(fixed_rate, notional, maturity, recovery_rate, default_prob, spread, num_simulations)
print("CVA:", cva)


CVA: 21.083251541633885
